In [177]:
import ee
import geemap.eefolium as geemap
import os
import requests
import math

In [178]:
def maskL8sr(image):
    # Bits 3 and 5 are cloud shadow and cloud, respectively.
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 5)
    # Get the pixel QA band.
    qa = image.select('pixel_qa')
    # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
                 .And(qa.bitwiseAnd(cloudsBitMask).eq(0))
    return image.updateMask(mask)

In [179]:
#MASKING FUNCTION FOR L5 AND L7
def cloudMaskL457(image):
    qa = image.select('pixel_qa')
    # If the cloud bit (5) is set and the cloud confidence (7) is high
    # or the cloud shadow bit is set (3), then it's a bad pixel.
    cloud = qa.bitwiseAnd(1 << 5) \
                  .And(qa.bitwiseAnd(1 << 7)) \
                  .Or(qa.bitwiseAnd(1 << 3))
    # Remove edge pixels that don't occur in all bands
    mask2 = image.mask().reduce(ee.Reducer.min())
    return image.updateMask(cloud.Not()).updateMask(mask2)

In [180]:
def generateLandsatImage(year, roi):
    if(year<1999):
        dataset_land = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR') \
                      .filterDate(str(year)+'-01-01', str(year)+'-12-31') \
                      .map(cloudMaskL457) \

        image_land = dataset_land.select(['B3', 'B2', 'B1']).median()
        land_vis['bands'] = ['B3', 'B2', 'B1']
    elif(year<2013):
        dataset_land = ee.ImageCollection('LANDSAT/LE07/C01/T1_SR') \
                      .filterDate(str(year)+'-01-01', str(year)+'-12-31') \
                      .map(cloudMaskL457) \

        image_land = dataset_land.select(['B3', 'B2', 'B1']).median()
        land_vis['bands'] = ['B3', 'B2', 'B1']
    else:
        dataset_land = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
                      .filterDate(str(year)+'-01-01', str(year)+'-12-31') \
                      .map(maskL8sr) \

        image_land = dataset_land.select(['B4', 'B3', 'B2']).median()
        land_vis['bands'] = ["B4","B3","B2"]
    url_land = image_land.getThumbURL(land_vis)
    return url_land

In [181]:
def generateWaterMapImage(year, roi):
    dataset_water = ee.ImageCollection("JRC/GSW1_2/YearlyHistory") \
                      .filterDate(str(year)+'-01-01', str(year)+'-12-31')
    image_water = dataset_water.select(['waterClass']).median()
    url_water = image_water.getThumbURL(water_vis)
    return url_water

In [182]:
def exportLocally(URL, fileName):
    r = requests.get(URL, stream=True)
    out_png = os.path.expanduser('~/Desktop/picture_samples/'+fileName+'.png')

    if r.status_code != 200:
        print('An error occurred while downloading.')
    else:
        with open(out_png, 'wb') as fd:
            for chunk in r.iter_content(chunk_size=1024):
                fd.write(chunk)
        print(fileName+ ' created!')

In [203]:
#Used for pixel dimension consitency since distance will always be the same
def calculatePointB(pointA, distance):
    
    R = 6378.1 #Radius of the Earth
    brng = math.radians(135) #Bearing is 135
    d = distance #Distance in km


    lat1_rad = math.radians(pointA[1]) #Current lat point converted to radians
    lon1_rad = math.radians(pointA[0]) #Current long point converted to radians

    lat2_cal = math.asin( math.sin(lat1_rad)*math.cos(d/R) + math.cos(lat1_rad)*math.sin(d/R)*math.cos(brng))

    lon2_cal = lon1_rad + math.atan2(math.sin(brng)*math.sin(d/R)*math.cos(lat1_rad),math.cos(d/R)-math.sin(lat1_rad)*math.sin(lat2_cal))

    lat2_deg = round(math.degrees(lat2_cal),4)
    lon2_deg = round(math.degrees(lon2_cal),4)
    print([lon2_deg,lat2_deg])
    return([lon2_deg,lat2_deg])

In [211]:
#IMAGE VISUALIZATION PARAMETERS
land_vis = {
    'gamma': 1.4,
    'max': 3000,
    'min': 0,
    'scale': 5,
    'region': roi,
    'format': 'png'
    }

water_vis = {
    'bands': ['waterClass'],
    'palette': ["cccccc","ffffff","99d9ea","0000ff"],
    'max': 3,
    'min': 0,
    'scale': 5,
    'region': roi,
    'format': 'png'
}

#REGION OF INTEREST 
#advice: gee uses [longitude, latitude], usually opposite
pointA=[29.0296, 54.7251]

setDistanceFromAtoB = 10 #in km

pointB= calculatePointB(pointA, setDistanceFromAtoB)

roi = ee.Geometry.Rectangle([pointA[0], pointA[1], pointB[0], pointB[1]])

###################
###DISTANCE INFO###
###################
# For width to be 10km set difference 0.10282
# For height to be 10km set difffernce 0.08993

#NAMING PURPOSES
latitude = str(pointA[0]).replace(".","-")
longitude = str(pointA[1]).replace(".","-")

[29.1394, 54.6615]


In [212]:
##CHANGE FOR NUMBER OF ITERATIONS
startYear = 2018 # MAX = 1985, MIN = 2019 ----- MAX 1984 IS POSSIBLE BUT DATA USUALLY NOT THAT GOOD
END_YEAR = 2020 # should be 2020 always, change only for testing

print(pointA, pointB)

for year in range(startYear,2020,1):
    fileName = latitude +'_'+ longitude +'_'+ str(year) +'_'
    landURL = generateLandsatImage(year, roi)
    waterURL = generateWaterMapImage(year, roi)
    exportLocally(landURL, fileName+'land')
    exportLocally(waterURL, fileName+'water')

[29.0296, 54.7251] [29.1394, 54.6615]
29-0296_54-7251_2018_land created!
29-0296_54-7251_2018_water created!
29-0296_54-7251_2019_land created!
29-0296_54-7251_2019_water created!
